In [5]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable
from PIL import Image

# Load the pretrained model
model = models.resnet18(pretrained=True)

# Use the model object to select the desired layer
layer = model._modules.get('avgpool')

# Set model to evaluation mode
model.eval()

# Image transforms
scaler = transforms.Resize((224, 224))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
to_tensor = transforms.ToTensor()

def get_vector(image_name):
    # 1. Load the image with Pillow library
    img = Image.open(image_name).convert("RGB")
    print(img)
    # 2. Create a PyTorch Variable with the transformed image
    t_img = Variable(normalize(to_tensor(scaler(img))).unsqueeze(0))
    # 3. Create a vector of zeros that will hold our feature vector
    #    The 'avgpool' layer has an output size of 512
#     my_embedding = torch.zeros(512)
    my_embedding = torch.zeros(1, 512, 1, 1)
    # 4. Define a function that will copy the output of a layer
    def copy_data(m, i, o):
        my_embedding.copy_(o.data)
    # 5. Attach that function to our selected layer
    h = layer.register_forward_hook(copy_data)
    # 6. Run the model on our transformed image
    model(t_img)
    # 7. Detach our copy function from the layer
    h.remove()
    # 8. Return the feature vector
    l1=[]
    for itr in my_embedding[0]:
        l1.append(float(itr[0][0]))
    return l1

In [6]:
v1=get_vector("/home/antpc/Downloads/ajio/aj_1.png")
v2=get_vector("/home/antpc/Downloads/ajio/myntra_1.png")

from numpy import dot
from numpy.linalg import norm

cos_sim = dot(v1, v2)/(norm(v1)*norm(v2))*100
print(cos_sim)

<PIL.Image.Image image mode=RGB size=594x734 at 0x7FC5D80B3790>
<PIL.Image.Image image mode=RGB size=458x612 at 0x7FC6051FF250>
87.66688549531727
